# Q3 KNN Classification 

In [744]:
from sklearn.neighbors import KDTree
import numpy
from data.data_utils import load_dataset
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pdb import set_trace
import time

## Useful Functions

In [745]:
np.random.seed(42)
def tie_breaker(values, count, x_idx, y_train):
    """
    x_idx = [ 91  53  56  25  26  96  1  2]
    y_train_ls = y_train[x_idx]
    y_train_ls = [111 222 33]
    values [1 2 3]  count [3 3 2]
    Given a list for indices of k-closest neighbours x_idx,
    where there are values each repeat for number of count
    return the value that is the closet
    """
    popular_v = []
    for i in range(len(count)):
        if count[i] == np.max(count):
            popular_v.append(values[i])
    # print(popular_v)
    for i in range(len(x_idx)):
        if y_train[i] in popular_v:
            
            # print("1", y_train[i])    
            return y_train[i]

# tie_breaker([1, 2, 3], [3, 3, 2], [9,8,7,6,5,4,3,2],[  3,3,1,1,1, 2,2,2])

In [746]:
def knn_classifier(x_train, y_train, x_test, k, metric):
    y_hat = np.empty((x_test.shape[0], 1), dtype=int)
    if metric == "cityblock":
        kd = KDTree(x_train, metric="cityblock")
    elif metric == "euclidean":
        kd = KDTree(x_train, metric="euclidean")
    else:
        pass
    # The index of each neighbor in self.data. i is the same shape as distance. Missing neighbors are indicated with self.n.
    idx = kd.query(x_test, k, return_distance=False, sort_results=False)
    # sample idx: [[ 76  53], [ 32  40], [ 26  96], [ 24  78], [ 10  85]]
    # there exists a closest neighbor with x_test value that ranks 53 in x_train
    # idx = idx[:,1].reshape(-1,1)
    # print("idx",idx)
    for i, x_test_i in enumerate(x_test):
        # print("y_train[idx[i]]", y_train[idx[i]])
        # find the vlaues and number of occurances for k-closest y_train values for a given x_test_i
        values, counts = np.unique(y_train[idx[i]], return_counts=True)
        # print(values, counts)
        if np.count_nonzero(counts == np.max(counts)) != 1:
            # a tie example: 6 closest, getting y_train = [[1] * 3, [2] * 3]
            # then 
            #print(k_nearest_neighbours[0])
            #print(y_train[k_nearest_neighbours[0]])
            # if there is a tie in the number of occurance
            y_hat[i, 0] = tie_breaker(values, counts, idx[i], y_train[idx[i]])
            # print("we have a tie for the most popular votes among k neighbours but we chose %d" %(y_hat[i, 0]))
        else:
            y_hat[i, 0] = values[np.argmax(counts)]
    #     print(idx)
    # y_test = vote[np.argmax(count)]
    # print(y_hat.shape)
    return y_hat

In [747]:
def get_accuracy(prediction, truth) -> float:
    """
    Find the fraction of correct predictions for classification problems
    """
    return np.mean(prediction == truth)

def knn_best_km(x_train, y_train, x_valid, y_valid, metrics, dataset, step=5):
    """
    Finding the best k and metric that maximizes accuracy
    """
    # Create random k value
    n = x_train.shape[0]
    min_k = dict(iris=1,  mnist_small=1)  # specify minimum k value to consider in the search
    max_k = dict(iris=np.sqrt(n), mnist_small=np.sqrt(n)) # specify maximum k value to consider in the search
    # Define a range of k
    k_vals = np.arange(min_k[dataset], max_k[dataset]+1, step, dtype=int)
    # k_vals = [1,6, ]
    # print(k_vals)
    acc_ls = []
    for m in metrics:
        acc_for_m = []
    # m = metrics[0]
        for k in k_vals:
    # k = 2
            y_valid_p = knn_classifier(x_train, y_train, x_valid, k=k, metric = m)
            acc = get_accuracy(y_valid_p, y_valid)
            print(m, k, acc)
            acc_for_m.append(acc)
        acc_ls.append(acc_for_m)
    print("acc_ls",acc_ls)
    acc_ls = np.asarray(acc_ls)
    best = np.unravel_index(np.argmax(acc_ls), acc_ls.shape)
    best_k = best[1]
    best_m = best[0]
    print(best)
    print("The highest accuracy occurs for k = %d with metric %s: %f"%(k_vals[best_k], metrics[best_m], acc_ls[best]))
    # best_m = acc_ls.min(axis=1)
    # print("best_k", k_vals[best_k], metrics[best_m])
    return k_vals[best_k], metrics[best_m], acc_ls[best]
    

## iris

In [748]:
x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset('iris')
# y_train.shape[1]
np.arange(y_train.shape[1]).reshape((1,-1)) 
# we don't care how many elements there are in the inner most dimension, but there needs to be 1 element in the outer most list.
# x_train.shape #(104, 4)
# y_train[32,40] #(104, 3)

array([[0, 1, 2]])

In [749]:
(y_train.shape[0], 1)

(104, 1)

In [750]:
y_train

array([[False,  True, False],
       [ True, False, False],
       [False,  True, False],
       [ True, False, False],
       [False, False,  True],
       [ True, False, False],
       [ True, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False,  True, False],
       [False,  True, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [ True, False, False],
       [False,  True, False],
       [False,  True, False],
       [ True, False, False],
       [False, False,  True],
       [False,  True, False],
       [ True, False, False],
       [False,  True, False],
       [False, False,  True],
       [False, False,  True],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [False, False,  True],
       [False,  True, False],
       [Fa

In [751]:
# np.tile(A, reps): Construct an array by repeating A the number of times given by reps.
# repeat [[0, 1, 2]] for (104, 1) times: make 104 rows and in each row we only have one copy of [0,1,2]
np.tile(np.arange(y_train.shape[1]).reshape((1,-1)), (y_train.shape[0], 1))

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0,

In [752]:
np.tile(np.arange(y_train.shape[1]).reshape((1,-1)), (y_train.shape[0], 1))[y_train]
# (104,3)[(104,3)] is like taking dot proudct
# [0, 1, 2] . [false, true, false] = 1 etc

array([1, 0, 1, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 1, 1, 0,
       2, 1, 0, 1, 2, 2, 0, 0, 0, 2, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 2, 2, 0, 0, 1, 2, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 2, 0,
       0, 0, 2, 1, 1, 1, 2, 2, 0, 2, 2, 1, 0, 1, 0, 1, 0, 0, 1, 1, 2, 0,
       0, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0])

In [753]:
y_train_encoded = np.tile(np.arange(y_train.shape[1]).reshape((1,-1)), (y_train.shape[0], 1))[y_train].reshape(-1,1)
y_valid_encoded = np.tile(np.arange(y_valid.shape[1]).reshape((1,-1)), (y_valid.shape[0], 1))[y_valid].reshape(-1,1)
y_test_encoded = np.tile(np.arange(y_test.shape[1]).reshape((1,-1)), (y_test.shape[0], 1))[y_test].reshape(-1,1)
y_train_encoded

array([[1],
       [0],
       [1],
       [0],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [1],
       [2],
       [2],
       [2],
       [0],
       [1],
       [1],
       [0],
       [2],
       [1],
       [0],
       [1],
       [2],
       [2],
       [0],
       [0],
       [0],
       [2],
       [1],
       [1],
       [2],
       [0],
       [1],
       [1],
       [2],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [2],
       [2],
       [0],
       [0],
       [1],
       [2],
       [1],
       [0],
       [1],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [0],
       [2],
       [0],
       [0],
       [0],
       [2],
       [1],
       [1],
       [1],
       [2],
       [2],
       [0],
       [2],
       [2],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
    

In [754]:
x_test.shape

(15, 4)

In [755]:
k1, m1, _ = knn_best_km(x_train, y_train_encoded, x_valid, y_valid_encoded, ["cityblock", "euclidean"], 'iris')
x = np.vstack([x_valid, x_train])
y = np.vstack([y_valid_encoded, y_train_encoded])
yp1 = knn_classifier(x, y, x_test, k1, m1)
print(get_accuracy(yp1, y_test_encoded))

cityblock 1 0.7741935483870968
cityblock 6 0.7419354838709677
cityblock 11 0.8709677419354839
euclidean 1 0.7741935483870968
euclidean 6 0.8709677419354839
euclidean 11 0.8709677419354839
acc_ls [[0.7741935483870968, 0.7419354838709677, 0.8709677419354839], [0.7741935483870968, 0.8709677419354839, 0.8709677419354839]]
(0, 2)
The highest accuracy occurs for k = 11 with metric cityblock: 0.870968
1.0


In [756]:
yp1

array([[2],
       [0],
       [0],
       [2],
       [0],
       [0],
       [1],
       [1],
       [2],
       [0],
       [2],
       [2],
       [1],
       [1],
       [1]])

## mnist small

In [757]:
x_train, x_valid, x_test,y_train, y_valid, y_test = load_dataset('mnist_small')
y_train_encoded = np.tile(np.arange(y_train.shape[1]).reshape((1,-1)), (y_train.shape[0], 1))[y_train].reshape(-1,1)
y_valid_encoded = np.tile(np.arange(y_valid.shape[1]).reshape((1,-1)), (y_valid.shape[0], 1))[y_valid].reshape(-1,1)
y_test_encoded = np.tile(np.arange(y_test.shape[1]).reshape((1,-1)), (y_test.shape[0], 1))[y_test].reshape(-1,1)

In [758]:
x_train.shape

(10000, 784)

In [759]:
k2, m2, _ = knn_best_km(x_train, y_train_encoded, x_valid, y_valid_encoded, ["cityblock", "euclidean"], 'mnist_small', step=20)
x = np.vstack([x_valid, x_train])
y = np.vstack([y_valid_encoded, y_train_encoded])
yp2 = knn_classifier(x, y, x_test, k2, m2)
print(get_accuracy(yp2, y_test_encoded))

cityblock 1 0.941
cityblock 21 0.911
cityblock 41 0.897
cityblock 61 0.877
cityblock 81 0.859
euclidean 1 0.95
euclidean 21 0.925
euclidean 41 0.915
euclidean 61 0.893
euclidean 81 0.886
acc_ls [[0.941, 0.911, 0.897, 0.877, 0.859], [0.95, 0.925, 0.915, 0.893, 0.886]]
(1, 0)
The highest accuracy occurs for k = 1 with metric euclidean: 0.950000
0.959
